Loading the json with the grade data:

In [1]:
import json
with open('grades.json', 'rb') as f:
    data = json.load(f)

Extracting the relevant information out of the json for one course:

In [2]:
def build_dict(course): 
        d = {
            'course_name_de': course['content']['achievementDto']['cpCourseLibDto']['courseTitle']['value'], 
            'course_name_en': course['content']['achievementDto']['cpCourseLibDto']['courseTitle']['translations']['translation'][1]['value'], 
            'ects': course['content']['achievementDto']['credits'], 
            'semester_code': course['content']['achievementDto']['semesterLibDto']['key'], 
            'semester_name_de': course['content']['achievementDto']['semesterLibDto']['semesterDesignation']['value'], 
            'semester_name_en': course['content']['achievementDto']['semesterLibDto']['semesterDesignation']
            ['translations']['translation'][1]['value'], 
            'semester_start_date': course['content']['achievementDto']['semesterLibDto']['startOfAcademicSemester']['value'], 
            'semester_end_date': course['content']['achievementDto']['semesterLibDto']['endOfAcademicSemester']['value'], 
            'grade_date': course['content']['achievementDto']['achievementDate']['value'], 
            'grade_name_de': course['content']['achievementDto']['gradeDto']['name']['value'], 
            'grade_name_en': course['content']['achievementDto']['gradeDto']['name']['translations']['translation'][1]['value'], 
        }
        try:
            d['id'] = course['content']['achievementDto']['cpCourseLibDto']['id']
            d['course_number'] = course['content']['achievementDto']['cpCourseLibDto']['courseNumber']['courseNumber']
            d['grade'] = course['content']['achievementDto']['gradeDto']['value']
        except KeyError:
            d['id'] = 'Not A Course' # Thesis, Kolloquium
            d['course_number'] = 'Not A Course'
            d['grade'] = course['content']['achievementDto']['gradeDto']['key'].replace(',', '.')
        return d

Creating a list of dicts, each dict containing the info for one course.

In [3]:
dicts = [build_dict(course) for course in data['resource'] if course['content']['achievementDto']['achievementStatusType'] == 'FINAL']

In [4]:
import pandas as pd


possible_grades = ['1.0', '1.3', '1.4', '1.7', '2.0', '2.3', '2.4', '2.7', '3.0', '3.3', '3.4', '3.7', '4.0', '4.3', '4.7', '5.0']
standard_possible_grades = ['1.0', '1.3', '1.7', '2.0', '2.3', '2.7', '3.0', '3.3', '3.7', '4.0', '4.3', '4.7', '5.0']
all_possible_grades = possible_grades + ['did_not_show_up']

for d in reversed(dicts): # iterating in reverse order so we can remove elements while iterating.
    # University regulation: written exams from first semester are weighted half the points.
    d['grade_weight'] = d['ects']
    if ('Discrete Structures' in d['course_name_en'] 
        or 'Introduction to Informatics' in d['course_name_en']
        or 'Computer Organization' in d['course_name_en']
       ):
        d['grade_weight'] >>= 1 # divide by 2 but leave as int (known to all be devisable by 2)
    # read the json file to a dict
    try:
        d.update((grade, 0) for grade in all_possible_grades) # All courses get all grades, also if 0 students that grade.
        with open(f'json_stats/{d["id"]}.json', 'rb') as f:
            j = json.load(f)
        ge = pd.DataFrame(j['gradeEntries'])
        groupby_cols = ['gradeShortName']
        if 'gradeCommentName' in ge:
            groupby_cols.append('gradeCommentName')
        se = ge.fillna('').groupby(groupby_cols).count()['id']
        se.index = se.index.map(''.join)  # From MultiIndex to normal index.
        se.index = se.index.str.replace(',', '.')  # From German format to normal float format.
        se.index = se.index.str.replace('5.0Nicht erschienen', 'did_not_show_up')
        se['5.0'] = se.get('5.0',0) + se.get('5.0Ungültig/Täuschung', 0)
        se['did_not_show_up'] = se.get('did_not_show_up',0) + se.get('QRücktritt mit anerkanntem Grund', 0) + se.get('NNicht erschienen', 0)
        se = se.drop([x for x in ['QRücktritt mit anerkanntem Grund', '5.0Ungültig/Täuschung', 'NNicht erschienen'] if x in se.index])
        d.update(se)
    except FileNotFoundError:
        print("No statistics file for ", d['course_name_en'])
        d[str(float(d['grade']))] = 1
    except KeyError:
        print("Empty statistics file for ", d['course_name_en'])
        d[str(float(d['grade']))] = 1



No statistics file for  Safe Execution of WebAssembly during Query Processing
Empty statistics file for  Bachelor's Colloquium


For each course, parse the grades out of its html file, and add to its dict:

In [5]:
%%script false --no-raise-error
# This magic prevents the execution of code in this cell.

# This is how I parsed the html files before the internal API started sending structured json responses with this data.

from bs4 import BeautifulSoup

possible_grades = ['1.0', '1.3', '1.4', '1.7', '2.0', '2.3', '2.4', '2.7', '3.0', '3.3', '3.4', '3.7', '4.0', '4.3', '4.7', '5.0']
standard_possible_grades = ['1.0', '1.3', '1.7', '2.0', '2.3', '2.7', '3.0', '3.3', '3.7', '4.0', '4.3', '4.7', '5.0']
all_possible_grades = possible_grades + ['did_not_show_up']

for d in reversed(dicts): # iterating in reverse order so we can remove elements while iterating.
    # University regulation: written exams from first semester are weighted half the points.
    d['grade_weight'] = d['ects']
    if ('Discrete Structures' in d['course_name_en'] 
        or 'Introduction to Informatics' in d['course_name_en']
        or 'Computer Organization' in d['course_name_en']
       ):
        d['grade_weight'] >>= 1 # divide by 2 but leave as int (known to all be devisable by 2)
    # read the html file to a string
    try:
        with open('stats/{}.html'.format(d['id']), 'rb') as f:
            html_doc = f.read()    
        soup = BeautifulSoup(html_doc, 'html.parser')
        # the data can be found in the titles of div objects with the class "kandcountbox"
        divs = soup.find_all('div', 'kandcountbox')
        titles = [div['title'] for div in divs]
        # A list of tuples (<grade>, <number of students>) e.g. ('1.0', 3)
        nums = [(ts[-1].split()[0], int(ts[-2].split()[0])) for t in titles if (ts := t.split(','))]
        d.update((grade, 0) for grade in all_possible_grades) # All courses get all grades, also if 0 students that grade.
        for i, t in enumerate(titles):
            if 'Nicht erschienen' in t: # Students who did not show up
                d['did_not_show_up'] = nums[i][1]
            elif '5.0' in t: # add up fails and cheats together.
                d['5.0'] += nums[i][1]
        # We already counted all the 5.0s and added them, so don't add again.
        d.update((tup for tup in nums if tup[0] != '5.0'))
    except FileNotFoundError:
        print("No statistics file for ", d['course_name_en'])
        dicts.remove(d)


Create a pandas dataframe with the data:

In [6]:

df = pd.DataFrame(dicts)
df['did_show_up'] = df[possible_grades].sum(axis=1)
df['numeric_grade'] = pd.to_numeric(df['grade'])
df['int_grade_X10'] = df['grade'].apply(lambda x: int((x.replace('.', '') + '0')[:2]))
df['5.0_with_noshows'] = df['5.0'] + df['did_not_show_up']
df['total_students'] = df['did_show_up'] + df['did_not_show_up']
df['grade'] = df['grade'].apply(lambda s: (s + '.0')[:3])
grades_with_noshows = possible_grades + ['5.0_with_noshows']
grades_with_noshows.remove('5.0')
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(df)

,course_name_de,course_name_en,ects,semester_code,semester_name_de,semester_name_en,semester_start_date,semester_end_date,grade_date,grade_name_de,grade_name_en,id,course_number,grade,grade_weight,1.0,1.3,1.4,1.7,2.0,2.3,2.4,2.7,3.0,3.3,3.4,3.7,4.0,4.3,4.7,5.0,did_not_show_up,did_show_up,numeric_grade,int_grade_X10,5.0_with_noshows,total_students
0,Bachelor-Kolloquium,Bachelor's Colloquium,3,22S,Sommersemester 2022,Summer semester 2022,2022-04-01,2022-09-30,2022-05-03T00:00:00,sehr gut,very good,950615303,IN2260,1.0,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1.0,10,0,1
1,Sichere Ausführung von WebAssembly während der...,Safe Execution of WebAssembly during Query Pro...,12,22S,Sommersemester 2022,Summer semester 2022,2022-04-01,2022-09-30,2022-04-19T00:00:00,sehr gut,very good,Not A Course,Not A Course,1.3,12,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1.3,13,0,1
2,"Technik, Natur und Gesellschaft","Technology, Nature, and Society",3,21W,Wintersemester 2021/22,Winter semester 2021/22,2021-10-01,2022-03-31,2022-02-07T00:00:00,sehr gut,very good,950581514,ED0179,1.3,3,37,24,0,21,11,10,0,3,3,1,0,0,0,0,0,0,0,110,1.3,13,0,110
3,Systems Programming in C++,Systems Programming in C++,10,21S,Sommersemester 2021,Summer semester 2021,2021-04-01,2021-09-30,2021-09-14T00:00:00,sehr gut,very good,950560131,IN4256,1.0,10,14,7,0,2,2,1,0,3,2,1,0,0,0,0,0,4,0,36,1.0,10,4,36
4,Compilerbau I,Compiler Construction I,5,21S,Sommersemester 2021,Summer semester 2021,2021-04-01,2021-09-30,2021-07-26T00:00:00,gut,good,950552753,IN2227,2.0,5,9,10,0,15,18,15,0,22,31,13,0,12,7,8,0,1,40,161,2.0,20,41,201
5,Statistik für BWL (Einführung mit R),Statistics for Business Administration,6,21S,Sommersemester 2021,Summer semester 2021,2021-04-01,2021-09-30,2021-07-22T00:00:00,befriedigend,satisfactory,950548250,MA9712M,2.7,6,11,23,0,28,59,103,0,125,114,97,0,70,35,39,14,34,68,752,2.7,27,102,820
6,Computergestützte Statistik,Computational Statistics,6,21S,Sommersemester 2021,Summer semester 2021,2021-04-01,2021-09-30,2021-07-20T00:00:00,ausreichend,sufficient,950549876,MA3402,4.0,6,3,3,0,9,14,10,0,9,14,12,0,10,13,18,19,29,72,163,4.0,40,101,235
7,Grundlagen der Künstlichen Intelligenz,Techniques in Artificial Intelligence,5,20W,Wintersemester 2020/21,Winter semester 2020/21,2020-10-01,2021-03-31,2021-03-01T00:00:00,befriedigend,satisfactory,950511694,IN2062,3.0,5,20,29,0,44,66,70,0,77,86,69,0,83,65,61,39,107,454,816,3.0,30,561,1270
8,Höhere Algorithmik,Advanced Algorithms,6,20W,Wintersemester 2020/21,Winter semester 2020/21,2020-10-01,2021-03-31,2021-02-23T00:00:00,befriedigend,satisfactory,950525218,IN2360,2.7,6,8,4,0,4,6,1,0,2,9,6,0,4,2,7,2,11,36,66,2.7,27,47,102
9,Blockkurs Spanisch A1,Intensive Course Spanish A1,3,20S,Sommersemester 2020,Summer semester 2020,2020-04-01,2020-09-30,2020-09-09T00:00:00,sehr gut,very good,950478816,SZ1201-1,1.0,3,3,7,0,6,0,0,0,0,0,0,0,0,0,0,0,0,0,16,1.0,10,0,16


In [7]:
int_grades = list(map(lambda x: int(x.replace('.', '')), possible_grades)) # '1.3' -> 13
df['mean'] = ((df[possible_grades] * int_grades).sum(axis=1) / df['did_show_up']) / 10
df['mean_with_noshows'] = ((df[grades_with_noshows] * int_grades).sum(axis=1) / df['total_students']) / 10
df

,course_name_de,course_name_en,ects,semester_code,semester_name_de,semester_name_en,semester_start_date,semester_end_date,grade_date,grade_name_de,...,4.7,5.0,did_not_show_up,did_show_up,numeric_grade,int_grade_X10,5.0_with_noshows,total_students,mean,mean_with_noshows
0,Bachelor-Kolloquium,Bachelor's Colloquium,3,22S,Sommersemester 2022,Summer semester 2022,2022-04-01,2022-09-30,2022-05-03T00:00:00,sehr gut,...,0,0,0,1,1.0,10,0,1,1.000000,1.000000
1,Sichere Ausführung von WebAssembly während der...,Safe Execution of WebAssembly during Query Pro...,12,22S,Sommersemester 2022,Summer semester 2022,2022-04-01,2022-09-30,2022-04-19T00:00:00,sehr gut,...,0,0,0,1,1.3,13,0,1,1.300000,1.300000
2,"Technik, Natur und Gesellschaft","Technology, Nature, and Society",3,21W,Wintersemester 2021/22,Winter semester 2021/22,2021-10-01,2022-03-31,2022-02-07T00:00:00,sehr gut,...,0,0,0,110,1.3,13,0,110,1.539091,1.539091
3,Systems Programming in C++,Systems Programming in C++,10,21S,Sommersemester 2021,Summer semester 2021,2021-04-01,2021-09-30,2021-09-14T00:00:00,sehr gut,...,0,4,0,36,1.0,10,4,36,1.950000,1.950000
4,Compilerbau I,Compiler Construction I,5,21S,Sommersemester 2021,Summer semester 2021,2021-04-01,2021-09-30,2021-07-26T00:00:00,gut,...,0,1,40,161,2.0,20,41,201,2.640373,3.109950
5,Statistik für BWL (Einführung mit R),Statistics for Business Administration,6,21S,Sommersemester 2021,Summer semester 2021,2021-04-01,2021-09-30,2021-07-22T00:00:00,befriedigend,...,14,34,68,752,2.7,27,102,820,2.986037,3.153049
6,Computergestützte Statistik,Computational Statistics,6,21S,Sommersemester 2021,Summer semester 2021,2021-04-01,2021-09-30,2021-07-20T00:00:00,ausreichend,...,19,29,72,163,4.0,40,101,235,3.557055,3.999149
7,Grundlagen der Künstlichen Intelligenz,Techniques in Artificial Intelligence,5,20W,Wintersemester 2020/21,Winter semester 2020/21,2020-10-01,2021-03-31,2021-03-01T00:00:00,befriedigend,...,39,107,454,816,3.0,30,561,1270,3.268137,3.887244
8,Höhere Algorithmik,Advanced Algorithms,6,20W,Wintersemester 2020/21,Winter semester 2020/21,2020-10-01,2021-03-31,2021-02-23T00:00:00,befriedigend,...,2,11,36,66,2.7,27,47,102,3.087879,3.762745
9,Blockkurs Spanisch A1,Intensive Course Spanish A1,3,20S,Sommersemester 2020,Summer semester 2020,2020-04-01,2020-09-30,2020-09-09T00:00:00,sehr gut,...,0,0,0,16,1.0,10,0,16,1.393750,1.393750


In [8]:
import numpy as np

We want to calculate the Z-Score of each grade. For a single course, where I got the grade $x$, the mean grade is $\mu$ and the standard deviation is $\sigma$, then $Z = \frac{x - \mu}{\sigma}$. We now calculate that simulataneously for each course.

We already have the mean and the grade of each course, so now we calculate the standard deviation. For a single course we used the (biased) standard deviation $$\sigma = \sqrt{\frac{\sum_{x \in X}(x-\mu)^2}{N}}$$ where $X$ is the the set of all grades in this one exam and $N$ is the total number of students.
Our data looks a bit different however. What we have is for every possible grade, the number of students with that grade, so our calculation will look like this: 
$$\sigma = \sqrt{\frac{\sum_{g \in G}n_g(g-\mu)^2}{N}}$$
Where $G$ is the set of possible grades ($1.0$, $1.3$, ...) and for a grade $g \in G$, $n_g$ is the number of students with that grade in this exam.

First, we create a matrix which has for each exam the row vector $(\ \ldots \ (g - \mu)^2 \   \ldots\ )_{g \in G}$ which contain the squared difference of every possible grade and the mean:

In [9]:
# The (x - \mu)^2 part of the calculation for the sd.
squared_differences = np.square(np.array([np.array(int_grades)/10,]*len(df)) - np.array([df['mean'],] * len(int_grades)).transpose())

We now multiply this matrix element-wise with our data, and get the matrix 
$(n_{g,i}(g-\mu)^2)_{g \in G, i \in E}$ where $E$ is the set of exams and $n_{g,i}$ is the number of students who get the grade $g$ in exam $i$. We then sum each row, and divide it by the total number of students who took that exam, which gives us a vector where every element is the complete term inside the squre root above, for one exam. We then just take the square root element-wise, and get a vector of the standard deviations of all exams, which we add to our DataFrame.

In [10]:
df['standard_deviation'] = np.sqrt(np.multiply(df[possible_grades], squared_differences).sum(axis=1) / df['did_show_up'])

In [11]:
df

,course_name_de,course_name_en,ects,semester_code,semester_name_de,semester_name_en,semester_start_date,semester_end_date,grade_date,grade_name_de,...,5.0,did_not_show_up,did_show_up,numeric_grade,int_grade_X10,5.0_with_noshows,total_students,mean,mean_with_noshows,standard_deviation
0,Bachelor-Kolloquium,Bachelor's Colloquium,3,22S,Sommersemester 2022,Summer semester 2022,2022-04-01,2022-09-30,2022-05-03T00:00:00,sehr gut,...,0,0,1,1.0,10,0,1,1.000000,1.000000,0.000000
1,Sichere Ausführung von WebAssembly während der...,Safe Execution of WebAssembly during Query Pro...,12,22S,Sommersemester 2022,Summer semester 2022,2022-04-01,2022-09-30,2022-04-19T00:00:00,sehr gut,...,0,0,1,1.3,13,0,1,1.300000,1.300000,0.000000
2,"Technik, Natur und Gesellschaft","Technology, Nature, and Society",3,21W,Wintersemester 2021/22,Winter semester 2021/22,2021-10-01,2022-03-31,2022-02-07T00:00:00,sehr gut,...,0,0,110,1.3,13,0,110,1.539091,1.539091,0.558748
3,Systems Programming in C++,Systems Programming in C++,10,21S,Sommersemester 2021,Summer semester 2021,2021-04-01,2021-09-30,2021-09-14T00:00:00,sehr gut,...,4,0,36,1.0,10,4,36,1.950000,1.950000,1.277693
4,Compilerbau I,Compiler Construction I,5,21S,Sommersemester 2021,Summer semester 2021,2021-04-01,2021-09-30,2021-07-26T00:00:00,gut,...,1,40,161,2.0,20,41,201,2.640373,3.109950,0.892785
5,Statistik für BWL (Einführung mit R),Statistics for Business Administration,6,21S,Sommersemester 2021,Summer semester 2021,2021-04-01,2021-09-30,2021-07-22T00:00:00,befriedigend,...,34,68,752,2.7,27,102,820,2.986037,3.153049,0.892316
6,Computergestützte Statistik,Computational Statistics,6,21S,Sommersemester 2021,Summer semester 2021,2021-04-01,2021-09-30,2021-07-20T00:00:00,ausreichend,...,29,72,163,4.0,40,101,235,3.557055,3.999149,1.175724
7,Grundlagen der Künstlichen Intelligenz,Techniques in Artificial Intelligence,5,20W,Wintersemester 2020/21,Winter semester 2020/21,2020-10-01,2021-03-31,2021-03-01T00:00:00,befriedigend,...,107,454,816,3.0,30,561,1270,3.268137,3.887244,1.130250
8,Höhere Algorithmik,Advanced Algorithms,6,20W,Wintersemester 2020/21,Winter semester 2020/21,2020-10-01,2021-03-31,2021-02-23T00:00:00,befriedigend,...,11,36,66,2.7,27,47,102,3.087879,3.762745,1.362875
9,Blockkurs Spanisch A1,Intensive Course Spanish A1,3,20S,Sommersemester 2020,Summer semester 2020,2020-04-01,2020-09-30,2020-09-09T00:00:00,sehr gut,...,0,0,16,1.0,10,0,16,1.393750,1.393750,0.260933


Once we have all that, we just do the $\frac{x-\mu}{\sigma}$ calculation for each exam.

Of course we do it with numpy's vectorized operations.

In [12]:
df['z'] = (df['numeric_grade'] - df['mean']) / df['standard_deviation']
df

,course_name_de,course_name_en,ects,semester_code,semester_name_de,semester_name_en,semester_start_date,semester_end_date,grade_date,grade_name_de,...,did_not_show_up,did_show_up,numeric_grade,int_grade_X10,5.0_with_noshows,total_students,mean,mean_with_noshows,standard_deviation,z
0,Bachelor-Kolloquium,Bachelor's Colloquium,3,22S,Sommersemester 2022,Summer semester 2022,2022-04-01,2022-09-30,2022-05-03T00:00:00,sehr gut,...,0,1,1.0,10,0,1,1.000000,1.000000,0.000000,NaN
1,Sichere Ausführung von WebAssembly während der...,Safe Execution of WebAssembly during Query Pro...,12,22S,Sommersemester 2022,Summer semester 2022,2022-04-01,2022-09-30,2022-04-19T00:00:00,sehr gut,...,0,1,1.3,13,0,1,1.300000,1.300000,0.000000,NaN
2,"Technik, Natur und Gesellschaft","Technology, Nature, and Society",3,21W,Wintersemester 2021/22,Winter semester 2021/22,2021-10-01,2022-03-31,2022-02-07T00:00:00,sehr gut,...,0,110,1.3,13,0,110,1.539091,1.539091,0.558748,-0.427905
3,Systems Programming in C++,Systems Programming in C++,10,21S,Sommersemester 2021,Summer semester 2021,2021-04-01,2021-09-30,2021-09-14T00:00:00,sehr gut,...,0,36,1.0,10,4,36,1.950000,1.950000,1.277693,-0.743527
4,Compilerbau I,Compiler Construction I,5,21S,Sommersemester 2021,Summer semester 2021,2021-04-01,2021-09-30,2021-07-26T00:00:00,gut,...,40,161,2.0,20,41,201,2.640373,3.109950,0.892785,-0.717275
5,Statistik für BWL (Einführung mit R),Statistics for Business Administration,6,21S,Sommersemester 2021,Summer semester 2021,2021-04-01,2021-09-30,2021-07-22T00:00:00,befriedigend,...,68,752,2.7,27,102,820,2.986037,3.153049,0.892316,-0.320556
6,Computergestützte Statistik,Computational Statistics,6,21S,Sommersemester 2021,Summer semester 2021,2021-04-01,2021-09-30,2021-07-20T00:00:00,ausreichend,...,72,163,4.0,40,101,235,3.557055,3.999149,1.175724,0.376742
7,Grundlagen der Künstlichen Intelligenz,Techniques in Artificial Intelligence,5,20W,Wintersemester 2020/21,Winter semester 2020/21,2020-10-01,2021-03-31,2021-03-01T00:00:00,befriedigend,...,454,816,3.0,30,561,1270,3.268137,3.887244,1.130250,-0.237237
8,Höhere Algorithmik,Advanced Algorithms,6,20W,Wintersemester 2020/21,Winter semester 2020/21,2020-10-01,2021-03-31,2021-02-23T00:00:00,befriedigend,...,36,66,2.7,27,47,102,3.087879,3.762745,1.362875,-0.284603
9,Blockkurs Spanisch A1,Intensive Course Spanish A1,3,20S,Sommersemester 2020,Summer semester 2020,2020-04-01,2020-09-30,2020-09-09T00:00:00,sehr gut,...,0,16,1.0,10,0,16,1.393750,1.393750,0.260933,-1.509009


In [13]:
# Now compute Z-score with noshows:
squared_differences = np.square(np.array([np.array(int_grades)/10,]*len(df)) - np.array([df['mean_with_noshows'],] * len(int_grades)).transpose())
df['standard_deviation_with_noshows'] = np.sqrt(np.multiply(df[grades_with_noshows], squared_differences).sum(axis=1) / df['total_students'])
df['z_with_noshows'] = (df['numeric_grade'] - df['mean_with_noshows']) / df['standard_deviation_with_noshows']
df

,course_name_de,course_name_en,ects,semester_code,semester_name_de,semester_name_en,semester_start_date,semester_end_date,grade_date,grade_name_de,...,numeric_grade,int_grade_X10,5.0_with_noshows,total_students,mean,mean_with_noshows,standard_deviation,z,standard_deviation_with_noshows,z_with_noshows
0,Bachelor-Kolloquium,Bachelor's Colloquium,3,22S,Sommersemester 2022,Summer semester 2022,2022-04-01,2022-09-30,2022-05-03T00:00:00,sehr gut,...,1.0,10,0,1,1.000000,1.000000,0.000000,NaN,0.000000,NaN
1,Sichere Ausführung von WebAssembly während der...,Safe Execution of WebAssembly during Query Pro...,12,22S,Sommersemester 2022,Summer semester 2022,2022-04-01,2022-09-30,2022-04-19T00:00:00,sehr gut,...,1.3,13,0,1,1.300000,1.300000,0.000000,NaN,0.000000,NaN
2,"Technik, Natur und Gesellschaft","Technology, Nature, and Society",3,21W,Wintersemester 2021/22,Winter semester 2021/22,2021-10-01,2022-03-31,2022-02-07T00:00:00,sehr gut,...,1.3,13,0,110,1.539091,1.539091,0.558748,-0.427905,0.558748,-0.427905
3,Systems Programming in C++,Systems Programming in C++,10,21S,Sommersemester 2021,Summer semester 2021,2021-04-01,2021-09-30,2021-09-14T00:00:00,sehr gut,...,1.0,10,4,36,1.950000,1.950000,1.277693,-0.743527,1.277693,-0.743527
4,Compilerbau I,Compiler Construction I,5,21S,Sommersemester 2021,Summer semester 2021,2021-04-01,2021-09-30,2021-07-26T00:00:00,gut,...,2.0,20,41,201,2.640373,3.109950,0.892785,-0.717275,1.235302,-0.898526
5,Statistik für BWL (Einführung mit R),Statistics for Business Administration,6,21S,Sommersemester 2021,Summer semester 2021,2021-04-01,2021-09-30,2021-07-22T00:00:00,befriedigend,...,2.7,27,102,820,2.986037,3.153049,0.892316,-0.320556,1.019147,-0.444537
6,Computergestützte Statistik,Computational Statistics,6,21S,Sommersemester 2021,Summer semester 2021,2021-04-01,2021-09-30,2021-07-20T00:00:00,ausreichend,...,4.0,40,101,235,3.557055,3.999149,1.175724,0.376742,1.183755,0.000719
7,Grundlagen der Künstlichen Intelligenz,Techniques in Artificial Intelligence,5,20W,Wintersemester 2020/21,Winter semester 2020/21,2020-10-01,2021-03-31,2021-03-01T00:00:00,befriedigend,...,3.0,30,561,1270,3.268137,3.887244,1.130250,-0.237237,1.228703,-0.722098
8,Höhere Algorithmik,Advanced Algorithms,6,20W,Wintersemester 2020/21,Winter semester 2020/21,2020-10-01,2021-03-31,2021-02-23T00:00:00,befriedigend,...,2.7,27,47,102,3.087879,3.762745,1.362875,-0.284603,1.427182,-0.744646
9,Blockkurs Spanisch A1,Intensive Course Spanish A1,3,20S,Sommersemester 2020,Summer semester 2020,2020-04-01,2020-09-30,2020-09-09T00:00:00,sehr gut,...,1.0,10,0,16,1.393750,1.393750,0.260933,-1.509009,0.260933,-1.509009


In [14]:
# inclusive definition of percentile
df['percentile'] = (np.multiply(np.array([np.array(int_grades),]*len(df)) < np.array([df['int_grade_X10'],] * len(int_grades)).transpose(), df[possible_grades]).sum(axis=1) / df['did_show_up']) * 100

In [15]:
# calculate percentile including noshows
df['percentile_with_noshows'] = (np.multiply(np.array([np.array(int_grades),]*len(df)) < np.array([df['int_grade_X10'],] * len(int_grades)).transpose(), df[grades_with_noshows]).sum(axis=1) / df['total_students']) * 100
df

,course_name_de,course_name_en,ects,semester_code,semester_name_de,semester_name_en,semester_start_date,semester_end_date,grade_date,grade_name_de,...,5.0_with_noshows,total_students,mean,mean_with_noshows,standard_deviation,z,standard_deviation_with_noshows,z_with_noshows,percentile,percentile_with_noshows
0,Bachelor-Kolloquium,Bachelor's Colloquium,3,22S,Sommersemester 2022,Summer semester 2022,2022-04-01,2022-09-30,2022-05-03T00:00:00,sehr gut,...,0,1,1.000000,1.000000,0.000000,NaN,0.000000,NaN,0.000000,0.000000
1,Sichere Ausführung von WebAssembly während der...,Safe Execution of WebAssembly during Query Pro...,12,22S,Sommersemester 2022,Summer semester 2022,2022-04-01,2022-09-30,2022-04-19T00:00:00,sehr gut,...,0,1,1.300000,1.300000,0.000000,NaN,0.000000,NaN,0.000000,0.000000
2,"Technik, Natur und Gesellschaft","Technology, Nature, and Society",3,21W,Wintersemester 2021/22,Winter semester 2021/22,2021-10-01,2022-03-31,2022-02-07T00:00:00,sehr gut,...,0,110,1.539091,1.539091,0.558748,-0.427905,0.558748,-0.427905,33.636364,33.636364
3,Systems Programming in C++,Systems Programming in C++,10,21S,Sommersemester 2021,Summer semester 2021,2021-04-01,2021-09-30,2021-09-14T00:00:00,sehr gut,...,4,36,1.950000,1.950000,1.277693,-0.743527,1.277693,-0.743527,0.000000,0.000000
4,Compilerbau I,Compiler Construction I,5,21S,Sommersemester 2021,Summer semester 2021,2021-04-01,2021-09-30,2021-07-26T00:00:00,gut,...,41,201,2.640373,3.109950,0.892785,-0.717275,1.235302,-0.898526,21.118012,16.915423
5,Statistik für BWL (Einführung mit R),Statistics for Business Administration,6,21S,Sommersemester 2021,Summer semester 2021,2021-04-01,2021-09-30,2021-07-22T00:00:00,befriedigend,...,102,820,2.986037,3.153049,0.892316,-0.320556,1.019147,-0.444537,29.787234,27.317073
6,Computergestützte Statistik,Computational Statistics,6,21S,Sommersemester 2021,Summer semester 2021,2021-04-01,2021-09-30,2021-07-20T00:00:00,ausreichend,...,101,235,3.557055,3.999149,1.175724,0.376742,1.183755,0.000719,51.533742,35.744681
7,Grundlagen der Künstlichen Intelligenz,Techniques in Artificial Intelligence,5,20W,Wintersemester 2020/21,Winter semester 2020/21,2020-10-01,2021-03-31,2021-03-01T00:00:00,befriedigend,...,561,1270,3.268137,3.887244,1.130250,-0.237237,1.228703,-0.722098,37.500000,24.094488
8,Höhere Algorithmik,Advanced Algorithms,6,20W,Wintersemester 2020/21,Winter semester 2020/21,2020-10-01,2021-03-31,2021-02-23T00:00:00,befriedigend,...,47,102,3.087879,3.762745,1.362875,-0.284603,1.427182,-0.744646,34.848485,22.549020
9,Blockkurs Spanisch A1,Intensive Course Spanish A1,3,20S,Sommersemester 2020,Summer semester 2020,2020-04-01,2020-09-30,2020-09-09T00:00:00,sehr gut,...,0,16,1.393750,1.393750,0.260933,-1.509009,0.260933,-1.509009,0.000000,0.000000


In [16]:
# mean Z score
mean_z_score = np.multiply(df['z'], df['grade_weight']).sum() / df['grade_weight'].sum()
print(mean_z_score)
mean_z_score_with_noshows = np.multiply(df['z_with_noshows'], df['grade_weight']).sum() / df['grade_weight'].sum()
print(mean_z_score_with_noshows)

-0.8198824179720919
-0.9725575675566105


In [17]:
avg_grade = np.multiply(df['numeric_grade'], df['grade_weight']).sum() / df['grade_weight'].sum()
print(avg_grade)

1.8571428571428574


In [18]:
# Percentile is weighted by the grade weight and by the number of students in the course.
weights = np.multiply(df['did_show_up'], df['grade_weight'])
avg_percentile = np.multiply(df['percentile'], weights).sum() / weights.sum()
print(avg_percentile)
weights = np.multiply(df['total_students'], df['grade_weight'])
avg_percentile_with_noshows = np.multiply(df['percentile_with_noshows'], weights).sum() / weights.sum()
print(avg_percentile_with_noshows)

13.647289957240263
11.479537280062214


In [19]:
total_ects = df['ects'].sum()

In [20]:
df.fillna(0, inplace=True)
df.sort_values(by=['grade','ects', 'percentile', 'z', 'semester_start_date', 'course_name_en'], 
               ascending=[True, False, True, True, True, True], inplace=True, )
df

,course_name_de,course_name_en,ects,semester_code,semester_name_de,semester_name_en,semester_start_date,semester_end_date,grade_date,grade_name_de,...,5.0_with_noshows,total_students,mean,mean_with_noshows,standard_deviation,z,standard_deviation_with_noshows,z_with_noshows,percentile,percentile_with_noshows
3,Systems Programming in C++,Systems Programming in C++,10,21S,Sommersemester 2021,Summer semester 2021,2021-04-01,2021-09-30,2021-09-14T00:00:00,sehr gut,...,4,36,1.950000,1.950000,1.277693,-0.743527,1.277693,-0.743527,0.000000,0.000000
11,Einführung in die Theoretische Informatik,Introduction to Theory of Computation,8,20S,Sommersemester 2020,Summer semester 2020,2020-04-01,2020-09-30,2020-08-13T00:00:00,sehr gut,...,316,857,3.642751,3.929405,1.219888,-2.166388,1.216849,-2.407369,0.000000,0.000000
25,Praktikum Grundlagen der Programmierung,Fundamentals of Programming (Exercises & Labor...,6,18W,Wintersemester 2018/19,Winter semester 2018/19,2018-10-01,2019-03-31,2019-02-20T00:00:00,sehr gut,...,255,1447,3.199444,3.208155,1.403789,-1.566791,1.405951,-1.570577,0.000000,0.000000
18,Grundlagen: Datenbanken,Fundamentals of Databases,6,19W,Wintersemester 2019/20,Winter semester 2019/20,2019-10-01,2020-03-31,2020-02-14T00:00:00,sehr gut,...,343,1216,2.860829,3.302385,1.199585,-1.551227,1.375342,-1.674046,0.000000,0.000000
19,Funktionale Programmierung und Verifikation,Functional Programming and Verification,5,19W,Wintersemester 2019/20,Winter semester 2019/20,2019-10-01,2020-03-31,2020-02-08T00:00:00,sehr gut,...,243,868,3.291712,3.551498,1.209361,-1.894978,1.271392,-2.006854,0.000000,0.000000
9,Blockkurs Spanisch A1,Intensive Course Spanish A1,3,20S,Sommersemester 2020,Summer semester 2020,2020-04-01,2020-09-30,2020-09-09T00:00:00,sehr gut,...,0,16,1.393750,1.393750,0.260933,-1.509009,0.260933,-1.509009,0.000000,0.000000
28,Deutsch im Bachelorstudium C1 - Informatik: Wi...,German for Bachelor's Students C1 - Informatic...,3,18W,Wintersemester 2018/19,Winter semester 2018/19,2018-10-01,2019-03-31,2019-02-06T00:00:00,sehr gut,...,2,25,1.126087,1.436000,0.301038,-0.418841,1.089910,-0.400033,0.000000,0.000000
0,Bachelor-Kolloquium,Bachelor's Colloquium,3,22S,Sommersemester 2022,Summer semester 2022,2022-04-01,2022-09-30,2022-05-03T00:00:00,sehr gut,...,0,1,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Sichere Ausführung von WebAssembly während der...,Safe Execution of WebAssembly during Query Pro...,12,22S,Sommersemester 2022,Summer semester 2022,2022-04-01,2022-09-30,2022-04-19T00:00:00,sehr gut,...,0,1,1.300000,1.300000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
16,Grundlagen: Betriebssysteme und Systemsoftware,Basic Principles: Operating Systems and System...,6,19W,Wintersemester 2019/20,Winter semester 2019/20,2019-10-01,2020-03-31,2020-02-24T00:00:00,sehr gut,...,198,852,3.021674,3.307277,1.202123,-1.432194,1.311459,-1.530568,6.584362,5.633803


Save the DataFrame data to a json:

In [21]:
df.to_json(path_or_buf='docs/js/grade_data.json', orient='records')

In [22]:
df[possible_grades + ['grade_weight']]

,1.0,1.3,1.4,1.7,2.0,2.3,2.4,2.7,3.0,3.3,3.4,3.7,4.0,4.3,4.7,5.0,grade_weight
3,14,7,0,2,2,1,0,3,2,1,0,0,0,0,0,4,10
11,42,15,0,21,23,35,0,33,35,57,0,40,96,68,76,135,8
25,197,67,0,60,77,81,0,93,98,108,0,97,78,102,134,248,6
18,98,3,49,71,91,18,74,108,88,44,38,73,42,42,34,92,6
19,39,23,0,40,57,50,0,55,67,74,0,69,59,39,53,111,5
9,3,7,0,6,0,0,0,0,0,0,0,0,0,0,0,0,3
28,18,3,0,1,0,1,0,0,0,0,0,0,0,0,0,0,3
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3
1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12
16,48,39,0,49,60,60,0,92,65,79,0,36,23,64,39,75,6


In [23]:
# multiply the number of students that got each grade, with the weight of that exam.
weighted_nums = (df[possible_grades].transpose() * df['grade_weight']).transpose()
weighted_nums

,1.0,1.3,1.4,1.7,2.0,2.3,2.4,2.7,3.0,3.3,3.4,3.7,4.0,4.3,4.7,5.0
3,140,70,0,20,20,10,0,30,20,10,0,0,0,0,0,40
11,336,120,0,168,184,280,0,264,280,456,0,320,768,544,608,1080
25,1182,402,0,360,462,486,0,558,588,648,0,582,468,612,804,1488
18,588,18,294,426,546,108,444,648,528,264,228,438,252,252,204,552
19,195,115,0,200,285,250,0,275,335,370,0,345,295,195,265,555
9,9,21,0,18,0,0,0,0,0,0,0,0,0,0,0,0
28,54,9,0,3,0,3,0,0,0,0,0,0,0,0,0,0
0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,12,0,0,0,0,0,0,0,0,0,0,0,0,0,0
16,288,234,0,294,360,360,0,552,390,474,0,216,138,384,234,450


In [24]:
sums = weighted_nums.sum()
sums

1.0     6195
1.3     3142
1.4      492
1.7     4646
2.0     6120
2.3     6091
2.4      648
2.7     7175
3.0     7158
3.3     6744
3.4      756
3.7     6830
4.0     5061
4.3     8871
4.7     6230
5.0    10371
dtype: int64

In [25]:
denominator = sums.sum()
denominator

86530

In [26]:
mean = (sums * int_grades).sum() / (denominator * 10)
mean

3.20281405292962

In [27]:
grades_array = np.array(int_grades)/10
squared_diffs = np.square(grades_array - mean)
squared_diffs

array([4.85238975, 3.62070132, 3.25013851, 2.25845008, 1.44676165,
       0.81507321, 0.6445104 , 0.25282197, 0.04113354, 0.00944511,
       0.0388823 , 0.24719387, 0.63550543, 1.203817  , 2.24156576,
       3.22987733])

In [28]:
# element wise multiplication, to weight each diff with its grade weight, and count it for every student with that grade
weighted_squared_diffs = np.multiply(sums, squared_diffs)
weighted_squared_diffs

1.0    30060.554512
1.3    11376.243548
1.4     1599.068147
1.7    10492.759061
2.0     8854.181273
2.3     4964.610947
2.4      417.642742
2.7     1813.997648
3.0      294.433880
3.3       63.697810
3.4       29.395017
3.7     1688.334105
4.0     3216.293003
4.3    10679.060629
4.7    13964.954685
5.0    33497.057772
dtype: float64

In [29]:
from math import sqrt
sd = sqrt(weighted_squared_diffs.sum() / denominator)
sd

1.2398310967553547

In [30]:
total_z = (avg_grade - mean) / sd
total_z

-1.0853665465468578

This "total Z-Score" is calculated to take into account the number of students in each course, unlike when just taking an average (weighted for credits but not for students) of the Z-Scores of all courses. This result is less affected by courses with a small amount of participants. This might make sense if you are trying to compare me to the total population of computer science students. A course with a small number of participants could be like a study with a small number of participants. It is less likely to provide a selection representative of the entire population. In the "Mean Z-Score" we only weight courses by their ECTS, here we also take the number of participants into account.

In [31]:
# Calculate total Z again with noshows:
weighted_nums_noshows = (df[grades_with_noshows].transpose() * df['grade_weight']).transpose()
sums_noshows = weighted_nums_noshows.sum()
denominator_noshows = sums_noshows.sum()
mean_noshows = (sums_noshows * int_grades).sum() / (denominator_noshows * 10)
grades_array = np.array(int_grades)/10
squared_diffs_noshows = np.square(grades_array - mean_noshows)
weighted_squared_diffs_noshows = np.multiply(sums_noshows, squared_diffs_noshows)
sd_noshows = sqrt(weighted_squared_diffs_noshows.sum() / denominator_noshows)
total_z_with_noshows = (avg_grade - mean_noshows) / sd_noshows
total_z_with_noshows

-1.2420861804844698

In [32]:
with open('docs/js/agg_data.json', 'w') as agg_f:
    json.dump({
        'mean_z_score': total_z,
        'mean_z_score_with_noshows': total_z_with_noshows,
        'avg_grade': avg_grade,
        'avg_percentile': avg_percentile,
        'avg_percentile_with_noshows': avg_percentile_with_noshows,
        'total_ects': int(total_ects),
    }, agg_f)

In [33]:
def aggregate_semester(group_frame):
    d = {}
    d['Number of Courses'] = len(group_frame.index)
    d['Total ECTS'] = group_frame['ects'].sum()
    d['My Mean Grade'] = np.average(group_frame.numeric_grade, weights=group_frame.grade_weight)
    d['My Mean Z-Score'] = np.average(group_frame.z, weights=np.multiply(group_frame.grade_weight, group_frame.did_show_up))
    d['My Mean Percentile'] = np.average(group_frame.percentile, weights=np.multiply(group_frame.grade_weight, group_frame.did_show_up))
    # Using dtype object to prevent int to float conversion. 
    # It's not that bad because no calculation will be mad on the series in that state,
    # And once back in the dataframe, each column gets a specific type
    return pd.Series(d, dtype=object)


semester_data = df.groupby(['semester_start_date', 'semester_code'] # include code for display, date for sort
                          ).apply(aggregate_semester
                                 ).sort_values('semester_start_date' # probably already sorted, but to be safe.
                                              ).reset_index().drop('semester_start_date', axis=1
                                                                  ).fillna(0)
semester_data

,semester_code,Number of Courses,Total ECTS,My Mean Grade,My Mean Z-Score,My Mean Percentile
0,18W,5,31,2.045000,-1.151321,10.451378
1,19S,4,28,1.850000,-0.997573,15.431123
2,19W,5,31,1.451613,-1.448749,6.233766
3,20S,6,37,1.832432,-1.486196,10.250077
4,20W,2,11,2.836364,-0.241428,37.265416
5,21S,4,27,2.229630,-0.288951,30.323065
6,21W,1,3,1.300000,-0.427905,33.636364
7,22S,2,15,1.240000,0.000000,0.000000


In [34]:
# Check that the types make sense:
semester_data.dtypes

semester_code          object
Number of Courses       int64
Total ECTS              int64
My Mean Grade         float64
My Mean Z-Score       float64
My Mean Percentile    float64
dtype: object

In [35]:
with open('docs/js/semester_data.json', 'w') as semester_f:
    json.dump(semester_data.to_dict('list'), semester_f)

In [36]:
my_grade_distribution = df[['grade', 'grade_weight']].groupby('grade').sum()
my_grade_distribution['grade_weight'] = my_grade_distribution['grade_weight'] / my_grade_distribution['grade_weight'].sum()
empty_frame = pd.DataFrame(0, index=standard_possible_grades, columns=['grade_weight'], dtype=my_grade_distribution.dtypes[0])
complete_distribution = pd.merge(my_grade_distribution, empty_frame, left_index=True, right_index=True, how='outer', sort=True).fillna(0)['grade_weight_x']
complete_distribution

1.0    0.261905
1.3    0.160714
1.7    0.154762
2.0    0.148810
2.3    0.000000
2.7    0.160714
3.0    0.053571
3.3    0.000000
3.7    0.023810
4.0    0.035714
4.3    0.000000
4.7    0.000000
5.0    0.000000
Name: grade_weight_x, dtype: float64

In [37]:
with open('docs/js/personal_distribution.json', 'w') as dist_f:
    json.dump(list(complete_distribution), dist_f)